In [1]:
import pandas as pd
import numpy as np
import os
import csv
import ast
import matplotlib.pyplot as plt
from collections import Counter
import geopandas as gpd
import h5py
from datetime import datetime, timedelta
import sys
# import mobility as mbl
from python_mobility import mobility_matrix_extract_modzcta as mm
from python_mobility import parameters as mp

my package for mobility extraction has been imported!


In [2]:
mp.cluster_name

{0: 'Grocery&Pharmacies',
 1: 'Retails',
 2: 'Arts&Entertainment',
 3: 'Restaurants&Bars',
 4: 'Educations',
 5: 'Healthcares',
 6: 'others'}

In [3]:
save_dir = '../../../../../../../Volumes/Seagate_Qing/Safegraph_NYC/'

In [4]:
wk_ls = []
for month in mp.months[1:]:
    # mm.extract_mobility_matrix(month, save_dir)   
    wk_ls.extend(mp.ym_datesofweek['2020'+month])
for month in mp.months[:2]:  
    wk_ls.extend(mp.ym_datesofweek['2021'+month])

In [5]:
len(wk_ls[3:])

53

In [6]:
print (mp.ym_datesofweek['202002'])
print (mp.ym_datesofweek['202102'])

['03', '10', '17', '24']
['01', '08', '15', '22']


### becareful with different matching level

In [7]:
df_nyc_tract_sorted = pd.read_csv('../data/mappings/nyc_tract_modzcta_v2021.csv')
df_poi_mapping = pd.read_csv('../data/mappings/nyc_tract_modzcta_poi_v2021.csv')
# df_poi_mapping = pd.read_csv('../data/df_nyc_tract_modidx_v33.csv')

In [5]:
## because the tract mapping provided in https://www.huduser.gov/apps/public/uspscrosswalk/home
## the last digit is equal to zero while the mobility data has 
## example: 36061009900 has 36061009901, 36061009902 and 36061009903 in the mobility data, ignore it 
df_nyc_tract_sorted['digit_count'] = df_nyc_tract_sorted['tract'].apply(lambda x: len(str(x)))
print(df_nyc_tract_sorted['digit_count'].unique())
my_tract = df_nyc_tract_sorted['tract'].values
df_nyc_tract_sorted['mod_idx'] = df_nyc_tract_sorted['mod_idx'].astype(int)
# df_nyc_tract_sorted['tract_10'] = df_nyc_tract_sorted['tract'].apply(lambda x: int(str(x)[:-1]))
modzcta_mapping = df_nyc_tract_sorted[['tract','res_ratio','mod_idx']].copy()
# modzcta_mapping.rename(columns={'TRACT':'tract'},inplace=True)
modzcta_mapping.set_index('tract',inplace=True)
df_modzcta_mapping = modzcta_mapping.groupby(level=0).apply(lambda df: df if isinstance(df, pd.DataFrame) else df.to_frame().T)

poi_mapping = df_poi_mapping[['tract','mod_idx']].copy()
# poi_mapping.rename(columns={'TRACT':'tract'},inplace=True)
poi_mapping.set_index('tract',inplace=True)

[11]


In [6]:
set(df_nyc_tract_sorted['tract'].unique().tolist()) - set(df_poi_mapping['tract'].unique().tolist())

set()

In [8]:
num_cate = 7 #5
num_mod = 177
#['03','04','05','06','07','08','09','10','11','12']
cbgs_list = []
for y in ['2021']:
    print('year:',y)
    for m in ['01','02']:
        print('month:',m)
        week_start_dates = mp.ym_datesofweek[y+m]
        d0 = week_start_dates[0]
        start_date = datetime.strptime(y+m+d0, '%Y%m%d')
        num_wk = len(week_start_dates)
        ### some of the months have more than 4 weeks 
        dates = [(start_date + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(num_wk*7)]
        M_raw = np.zeros((7*num_wk,num_cate,num_mod,num_mod)) ## time, POI, destination, origins
        for week_idx in range(len(week_start_dates)): #len(week_start_dates)
            start_date = week_start_dates[week_idx]
            print('week',start_date)
            dates_idx = np.array([0,1,2,3,4,5,6]) + np.array([week_idx*7]*7)
            
            file_name = save_dir+y+'_'+m+'_'+start_date+'.csv'
            df_i = pd.read_csv(file_name)
            ## THERE IS a place zip code in ny, but all cbgs are in and from CA, delete it
            ## a fuel station 
            df_clean = mm.pre_process_df(df_i,mp.cluster,poi_mapping,11)
            cbgs_list.extend(df_clean['poi_tract'].values.tolist())
            
            df_m = pd.DataFrame()
            df_m['cluster_idx'] = df_clean['cluster_qing'].astype(int)
            df_m['i'] = df_clean['mod_idx'].astype(int)
            df_m['raw_visits_count'] = df_clean['raw_visit_counts']
            df_m['raw_visitors'] = df_clean['raw_visitor_counts']
            df_m['visits_by_day_list'] = df_clean['visits_by_day'].apply(lambda x: ast.literal_eval(x))
            df_m['visitor_home_cbgs'] = df_clean['visitor_home_cbgs'].apply(lambda x: x[1:-1].replace('"','').split(',')) ### remove the first and last character, and then split by comma
            # df_m[['cbg_list_visitor', 'num_non_ny_visitors','num_reassign_cbgs']] = df_m['visitor_home_cbgs'].apply(lambda x: mm.remove_CA_non_NY(x,11))
            df_m['cbg_list_visitor'] = df_m['visitor_home_cbgs'].apply(lambda x: mm.remove_CA_non_NY(x,11))
            ### there are some place only has visitors from nonNY states
            cross_states_index = df_m[df_m['cbg_list_visitor'].apply(lambda x: len(x)) < 1].index
            df_m = df_m.drop(cross_states_index).reset_index(drop=True)
            #### assign the loss visitors to the known cbgs
            df_m['assigned_cbg_list_visitor'] = df_m.apply(lambda x: mm.account_for_loss_visitors(x['cbg_list_visitor'], x['raw_visitors']),axis=1)

            df_m['multiplier'] = df_m.apply(lambda x: x['raw_visits_count']/x['raw_visitors'], axis=1) ### multiplier
            ### adjust visitors to number of visits, and then assign visits to 7 days
            # df_m['cbg_visits_real'] = df_m.apply(lambda x: mm.assign_visits_by_raw_visitors(x['cbg_list_visitor'], x['multiplier'],my_tract),axis=1)
            df_m['cbg_visits_real'] = df_m.apply(lambda x: mm.assign_visits_by_raw_visitors(x['assigned_cbg_list_visitor'], x['multiplier'],my_tract),axis=1)
            #df_m['modz_visits_list'] = df_m['cbg_visits_real'].apply(lambda x: mm.distribute_visitor_modzcta(x, df_modzcta_mapping))
            df_m['modz_visits_list'] = df_m['cbg_visits_real'].apply(lambda x: mm.map_modzcta_visitor_list(x, poi_mapping))
            df_m['vis_daily_matrix'] = df_m.apply(lambda x: mm.find_fips_dayvisits(x['modz_visits_list'], x['visits_by_day_list'], x['raw_visits_count']),axis=1)
            for j_data_idx in range(len(df_m)):
                df_j = df_m.iloc[[j_data_idx]]
                M_raw = mm.mobility_extract_per_poi(df_j, M_raw, dates_idx)
        # mm.store_data_to_hdf5(M_raw, dates, y, m, num_cate, num_mod, filepath='../Data/mobility_cate7_1to1/M_raw_{}{}.h5')
        # mm.store_data_to_hdf5(M_raw, dates, y, m, num_cate, num_mod, filepath='../Data/mobility_cate7_1tomore/M_raw_{}{}.h5')
        mm.store_data_to_hdf5(M_raw, dates, y, m, num_cate, num_mod, filepath='../Data/mobility_cate7_loss_visitors/M_raw_{}{}.h5')

year: 2021
month: 01
week 04
week 11
week 18
week 25
month: 02
week 01
week 08
week 15
week 22
